In [1]:
#Notebook taken and modified from https://nbviewer.org/url/arteagac.github.io/blog/lime_image.ipynb

import numpy as np
from keras.applications.imagenet_utils import decode_predictions
import skimage.io 
import skimage.segmentation
import copy
import sklearn
import sklearn.metrics
from sklearn.linear_model import LinearRegression
import os
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt

2024-02-05 17:19:51.233878: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
folder = 'pickle'
inner_folder = 'augmented_model'

with open(os.path.join(folder, inner_folder, 'model_pickle'), 'rb') as f:
    model = pickle.load(f)

2024-02-05 17:19:53.638688: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-05 17:19:53.647486: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-05 17:19:53.647666: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

In [3]:
with open(os.path.join(folder, inner_folder, 'confusion_data'), 'rb') as f:
    loaded_confusion_data = pickle.load(f)
    
class_names = loaded_confusion_data['class_labels']

In [4]:
def perturb_image(img, perturbation, segments):
    active_pixels = np.where(perturbation == 1)[0]
    mask = np.zeros(segments.shape)
    for active in active_pixels:
        mask[segments == active] = 1 
    perturbed_image = copy.deepcopy(img)
    perturbed_image = perturbed_image * mask[:, :, np.newaxis]
    
    # Normalize to the range [0, 1]
    perturbed_image = perturbed_image / 255.0
    return perturbed_image

In [13]:
num_perturb = 50

for image_class in os.listdir('Validation'):
    image_class_path = os.path.join('Validation', image_class)
    images = os.listdir(image_class_path)
    num_files = len(images) * 0.3 + 1
    for i, image_name in enumerate(images):
        image_path = os.path.join(image_class_path, image_name)
        image = skimage.io.imread(image_path)

        main_prediction = model.predict(np.expand_dims(image/255, 0), verbose = 0)

        superpixels = skimage.segmentation.quickshift(image, kernel_size=4,max_dist=200, ratio=0.2)
        num_superpixels = np.unique(superpixels).shape[0]
        perturbations = np.random.binomial(1, 0.5, size=(num_perturb, num_superpixels))

        predictions = []
            
        perturbed_images = np.array([perturb_image(image, pert, superpixels) for pert in perturbations])
        predictions = model.predict(perturbed_images, verbose=0)

        predictions = np.array(predictions)

        image = np.ones(num_superpixels)[np.newaxis,:] #Perturbation with all superpixels enabled 
        distances = sklearn.metrics.pairwise_distances(perturbations,image, metric='cosine').ravel()

        kernel_width = 0.25
        weights = np.sqrt(np.exp(-(distances**2)/kernel_width**2)) #Kernel function

        class_to_explain = np.argmax(main_prediction)
        simpler_model = LinearRegression()
        
        simpler_model.fit(X=perturbations, y=predictions[:, class_to_explain], sample_weight=weights)
        coeff = simpler_model.coef_[0]

EBO
EBO_21609.jpg
EBO_16789.jpg
EBO_08997.jpg
EBO_17538.jpg
EBO_18097.jpg
EBO_25391.jpg
EBO_10879.jpg
EBO_13206.jpg
EBO_15156.jpg


KeyboardInterrupt: 